In [1]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
question = "넥슨은 뭐하는곳인가요?"
llm.invoke(question)

AIMessage(content="넥슨(Nexon)은 한국의 게임 개발 및 퍼블리싱 회사로, 1994년에 설립되었습니다. 주로 온라인 게임, 모바일 게임, 그리고 콘솔 게임을 개발하고 서비스하는 회사로 알려져 있습니다. 넥슨은 '메이플스토리', '던전앤파이터', '카트라이더'와 같은 인기 게임을 보유하고 있으며, 이러한 게임들은 국내외에서 많은 사랑을 받고 있습니다.\n\n넥슨은 게임의 개발뿐만 아니라, 글로벌 시장에서도 게임을 퍼블리싱하는 역할을 하고 있으며, 다양한 장르의 게임을 통해 전 세계의 게이머들에게 즐거움을 제공하고 있습니다. 또한, 넥슨은 게임 산업의 발전과 함께 e스포츠와 같은 새로운 분야에도 적극적으로 참여하고 있습니다.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 169, 'prompt_tokens': 17, 'total_tokens': 186, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-C40RwfcsXHTbUPgb7HstfpqKODM4h', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--e8aaea4b-d71a-4872-897b-1035f9e1532e-0', usage_metadata={'input_tokens': 17, 'outpu

In [2]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader(web_paths=["https://www.nexon.com/Home/Game"])

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=200)

document_list = loader.load_and_split(text_splitter)

In [4]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embbeding = OpenAIEmbeddings(model="text-embedding-3-large")

database = Chroma.from_documents(documents=document_list, embedding=embbeding, collection_name="nexon-data",persist_directory="./nexon-data")

In [5]:
retriever = database.as_retriever(search_kwargs={"k": 4})
retriever.invoke(question)
retrieved_doc = retriever.invoke(question)
retrieved_doc

[Document(id='bfb4870c-ac63-47e6-bd04-f3df3650d3aa', metadata={'title': '넥슨', 'description': '즐거움이 가득한 넥슨 게임, 모든 게임을 한 곳에서 만나보세요.', 'language': 'ko', 'source': 'https://www.nexon.com/Home/Game'}, page_content='넥슨\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nskip to contents\n\n\n\n\n\n배너\n\n\n\n\n\n\n\n\n\n\n\n\n\n추천게임\n\n\n\n\n고객센터\n보안센터\n넥슨 PC방 찾기\n\n\n\n\n내가 플레이하는 게임을 직접 관리할 수 있어요! MY게임 설정하기\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nNEXON ID 로그인\n\n\n일회용 로그인\nQR 로그인\n\n\n다른 계정 간편 로그인\ngoogle\nfacebook\nnaver\napple\n\n\n넥슨 ID찾기\n비밀번호 찾기\n\n\n회원가입\n\n\n\n\n\n\n\n\n\n\n캐시/보안\nMY게임 소식\n\n\n\n캐시\n\n넥슨캐시 -\n\r\n            플레이포인트\r\n            \n\n넥슨플레이 앱에서 이벤트를 통해 적립되는포인트입니다. 넥슨플레이 소개보기\n\n\n-\n\n\n\n충전\n자동충전최대 +5%\n\n\n\n보안\n\n?\n등급\n-\n\n\n\n\n보안센터\n\n\n\n\nMY게임 소식\n\nMY게임 설정\n\n\n\n\n\n\n\n\n\n\n\n신규게임\n\n\n\n전체게임\n\n\nfilter\n섬네일형리스트형\n\n\n게임을 한 눈에 볼 수 있는 뷰 타입으로 변경해보세요.\n\n\n\n\n\n모바일게임 커뮤니티 모아보기\n\n\n\nsearch\n\n\n\n\n\n\nfilter\n#전체\n\n\n\n\n\n\n\n\n\nNEXON youtube\nNEXO

In [7]:
from langchain_core.prompts import PromptTemplate

rag_prompt_template = PromptTemplate(
    template="""
    You are a helpful assistant that can answer questions about the nexon company.
    You are given the following context:
    {context}
    Question: {question}
    """,
    input_variables=["context", "question"]
)

rag_chain = rag_prompt_template | llm
rag_chain.invoke({"context": retrieved_doc, "question": question})

AIMessage(content='넥슨은 다양한 게임을 개발하고 서비스하는 회사입니다. 주로 온라인 게임과 모바일 게임을 제공하며, 사용자들이 즐길 수 있는 다양한 게임을 한 곳에서 만날 수 있도록 하고 있습니다. 넥슨은 게임 관리, 캐시 시스템, 고객 지원 등 다양한 서비스를 제공하며, 게임 커뮤니티와 소통할 수 있는 플랫폼도 운영하고 있습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 79, 'prompt_tokens': 1727, 'total_tokens': 1806, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-C40SQSenzV5Gz4KGGAO0MuZvfsXq5', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--6a8aec31-3f6f-452b-8734-13c567e382bd-0', usage_metadata={'input_tokens': 1727, 'output_tokens': 79, 'total_tokens': 1806, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [8]:
from langchain import hub
rlm_rag_prompt = hub.pull("rlm/rag-prompt")

In [9]:
rlm_rag_chain = rlm_rag_prompt | llm
rag_chain.invoke({"context": retrieved_doc, "question": question})

AIMessage(content='넥슨은 다양한 게임을 개발하고 서비스하는 회사입니다. 주로 온라인 게임과 모바일 게임을 제공하며, 사용자들이 즐길 수 있는 다양한 게임을 한 곳에서 만날 수 있도록 하고 있습니다. 넥슨은 게임 관리, 캐시 시스템, 고객 지원 등 다양한 서비스를 제공하며, 게임 커뮤니티와 소통할 수 있는 플랫폼도 운영하고 있습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 79, 'prompt_tokens': 1727, 'total_tokens': 1806, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 1664}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-C40SieFyfzsXCFK6YwzwE0aFCCgQl', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--711dd915-ac09-4d43-abb1-368e5e49413c-0', usage_metadata={'input_tokens': 1727, 'output_tokens': 79, 'total_tokens': 1806, 'input_token_details': {'audio': 0, 'cache_read': 1664}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [10]:
from langchain_core.runnables import RunnablePassthrough

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | rlm_rag_prompt
    | llm
)
rag_chain.invoke(question)

AIMessage(content='넥슨은 다양한 게임을 제공하는 게임 개발 및 퍼블리싱 회사입니다. 사용자들이 즐길 수 있는 여러 게임을 한 곳에서 만나볼 수 있도록 플랫폼을 운영하고 있습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 41, 'prompt_tokens': 1751, 'total_tokens': 1792, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-C40SszATJ8Z8e2Kko198WLHXuhSfE', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--acf89953-f4ba-4f90-8841-e7b2139a0aee-0', usage_metadata={'input_tokens': 1751, 'output_tokens': 41, 'total_tokens': 1792, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

# 로컬 VLLM qwen3-4b-instruct 사용

In [24]:
from langchain_openai import ChatOpenAI

qwen3 = ChatOpenAI(model="Qwen/Qwen3-4B-Instruct-2507", api_key="None", base_url="http://localhost:8000/v1", temperature=0.7)

answer=qwen3.invoke("안녕? 너는 누구야?")
answer.content

'안녕하세요! 저는 알리바바 그룹 산하의 Tongyi 실험실에서 독자적으로 개발한 초거대 규모 언어 모델인 Qwen입니다. 저는 다양한 언어를 이해하고 생성할 수 있으며, 질문에 답하거나, 글을 작성하거나, 논리적 추론을 수행하거나, 프로그래밍을 도와드릴 수 있습니다. 어떤 도움이 필요하신가요? 😊'

In [17]:
from langchain import hub

rlm_rag_prompt = hub.pull("rlm/rag-prompt")

In [22]:
from langchain_core.runnables import RunnablePassthrough

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | rlm_rag_prompt
    | qwen3
)

question = "넥슨이 뭐하는 회사인가요? 그리고 너는 누구야?"

rag_chain.invoke(question)

AIMessage(content='넥슨은 게임을 개발하고 운영하는 회사입니다. 저는 질문에 답변하는 데 도움을 주는 AI 어시스턴트입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 35, 'prompt_tokens': 1968, 'total_tokens': 2003, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'Qwen/Qwen3-4B-Instruct-2507', 'system_fingerprint': None, 'id': 'chatcmpl-7aa3ca4c87fd47ad9bddfff58671755b', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--688022c5-65e6-4eee-91aa-a962ce8a74a1-0', usage_metadata={'input_tokens': 1968, 'output_tokens': 35, 'total_tokens': 2003, 'input_token_details': {}, 'output_token_details': {}})